In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr, mannwhitneyu
from scipy.stats import ttest_ind
import datetime as dt

Вы работаете продуктовым аналитиком в агрегаторе такси SkyTaxi, и ваша задача - оценить результаты АВ-теста.

Тест предполагает введение новой платежной механики, которая должна немного понизить цену на услугу. Тест проводится путем разделения всех просмотров цены на группы А и В.

ℹ️ Просмотр - это событие, когда клиент вводит в приложении точку В и видит отображенную цену заказа. После этого он может или конвертироваться в созданный заказ (если цена его устроит), или нет.
Ваша задача - исследовать результат и понять, влияет ли понижение цены на конверсию из просмотра в создание заказа.

Для выполнения задания воспользуйтесь Python и оформите результат как .ipynb файл (текстовые выводы оформить там же).

In [85]:
Dataset = pd. read_excel('Финальная первичная аттестация. Вариант 1. Python (1).xlsx', sheet_name = 'Dataset')
city_dict = pd.read_excel('Финальная первичная аттестация. Вариант 1. Python (1).xlsx', sheet_name = 'city_dict')

In [86]:
Dataset.head(10)

,id_view,time_view,id_group,id_city,nflag_order
0,10457162393,2023-01-17 13:56:06.118,0,3,1
1,10457152884,2023-01-30 17:38:18.629,1,1,1
2,10457162947,2023-01-07 14:06:22.689,1,1,1
3,10457197478,2023-01-02 11:02:58.189,0,3,0
4,10457176480,2023-01-04 22:56:19.240,0,1,1
5,10457147632,2023-01-28 09:05:01.472,0,2,0
6,10457135372,2023-01-15 10:03:58.011,0,3,0
7,10457171869,2023-01-26 21:24:13.151,1,4,1
8,10457189220,2023-01-29 11:11:42.648,1,1,1
9,10457195432,2023-01-18 11:37:15.581,1,2,1


In [ ]:
df_cities = region_dict.groupby('city').count().reset_index()

# 3.1 Импортируйте файл с данными в Jupyter Notebook и расшифруйте идентификаторы городов с помощью листа city_dict

In [88]:
city_dict.head(20)

,id_city,name_city
0,1,Москва
1,2,Санкт-Петербург
2,3,Новосибирск
3,4,Казань
4,5,Краснодар
5,6,Челябинск
6,7,Сочи
7,8,Нижний Новгород
8,9,Воронеж
9,10,Архангельск


# 3.2 Рассчитайте результаты АВ-теста и с помощью статистического критерия сделайте вывод

In [89]:
Dataset = Dataset.merge(city_dict[['id_city','name_city']], on='id_city', how='outer')

In [90]:
Dataset

,id_view,time_view,id_group,id_city,nflag_order,name_city
0,10457162393,2023-01-17 13:56:06.118,0,3,1,Новосибирск
1,10457197478,2023-01-02 11:02:58.189,0,3,0,Новосибирск
2,10457135372,2023-01-15 10:03:58.011,0,3,0,Новосибирск
3,10457162971,2023-01-02 09:39:31.283,0,3,1,Новосибирск
4,10457132942,2023-01-23 19:18:03.516,0,3,0,Новосибирск
...,...,...,...,...,...,...
21934,10457174004,2023-01-21 11:47:30.578,0,9,1,Воронеж
21935,10457130320,2023-01-24 02:09:49.169,0,9,0,Воронеж
21936,10457166866,2023-01-13 16:22:38.491,1,9,0,Воронеж
21937,10457186695,2023-01-28 02:21:49.007,0,9,1,Воронеж


In [36]:
print('сверяем количества разбиений по группам)')
print(Dataset.groupby('id_group').count()['id_view'].reset_index())

сверяем количества разбиений по группам)
   id_group  id_view
0         0    10826
1         1    11113


In [ ]:
print('сверяем количества разбиений по группам)')
print(Dataset.groupby('id_group').count()['id_view'].reset_index())

In [37]:
print('сверяем количества по тестовой перехода в покупку)')
print(Dataset[Dataset['id_group']==0].groupby('nflag_order').count()['id_view'].reset_index())

сверяем количества по тестовой перехода в покупку)
   nflag_order  id_view
0            0     4167
1            1     6659


In [38]:
print('сверяем количества по контрольной перехода в покупку)')
print(Dataset[Dataset['id_group']==1].groupby('nflag_order').count()['id_view'].reset_index())

сверяем количества по контрольной перехода в покупку)
   nflag_order  id_view
0            0     3603
1            1     7510


проверим есть ли разница между выборками

In [41]:
group_test = Dataset[Dataset['id_group']==1]
group_control = Dataset[Dataset['id_group']==0]

In [43]:
import pandas as pd
from scipy. stats import ttest_ind
t_stat, p_val = ttest_ind(group_test['nflag_order'], group_control['nflag_order'])

print('значение статистического китерия - ', t_stat)
print('P-значение (англ. P-value), p-уровень значимости, p-критерий (сила нулевой гипотезы)—', p_val)

# Вывод результата
if p_val < 0.05:
    if p_val > 0:
        print('тест лучше чем контроль')
        print('новая механика лучше старой')
    else:
        print('тест хуже чем контроль')
        print('старая механика лучше новой')
else:
    print('p_val =', p_val, ' > 0.05')
    print('Нет статистически значимой разницы между механиками')

значение статистического китерия -  9.41595134321716
P-значение (англ. P-value), p-уровень значимости, p-критерий (сила нулевой гипотезы)— 5.1365957325220535e-21
тест лучше чем контроль
новая механика лучше старой


# Задание 3.3
Запустите цикл, который пробежится по всем городам в датасете. Выявите те города, в которых есть статистически значимая разница между группами А и В. Создайте переменную типа "список", которая будет хранить эти города.

In [45]:
city_spisok = []
for i in Dataset['name_city'].unique():
    print('--------------------------------------------')
    print(i)
    t_stat, p_val = ttest_ind(group_test[group_test['name_city']==i]['nflag_order'], group_control[group_control['name_city']==i]['nflag_order'])
    print('значение статистического китерия - ', t_stat)
    print('P-значение (англ. P-value), p-уровень значимости, p-критерий (сила нулевой гипотезы)—', p_val)
    if p_val < 0.05:
        if p_val > 0:
            print('тест лучше чем контроль')
            print('новая механика лучше старой')
        else:
            print('тест хуже чем контроль')
            print('старая механика лучше новой')
        city_spisok.append(i)
    else:
        print('p_val =', p_val, ' > 0.05')
        print('Нет статистически значимой разницы между старой и новой механиками')
    print('--------------------------------------------')    

--------------------------------------------
Новосибирск
значение статистического китерия -  1.4827891417672616
P-значение (англ. P-value), p-уровень значимости, p-критерий (сила нулевой гипотезы)— 0.13822636677161423
p_val = 0.13822636677161423  > 0.05
Нет статистически значимой разницы между старой и новой механиками
--------------------------------------------
--------------------------------------------
Москва
значение статистического китерия -  7.410981603370934
P-значение (англ. P-value), p-уровень значимости, p-критерий (сила нулевой гипотезы)— 1.368947706664987e-13
тест лучше чем контроль
новая механика лучше старой
--------------------------------------------
--------------------------------------------
Санкт-Петербург
значение статистического китерия -  5.220275988832596
P-значение (англ. P-value), p-уровень значимости, p-критерий (сила нулевой гипотезы)— 1.8595141124883944e-07
тест лучше чем контроль
новая механика лучше старой
--------------------------------------------
--

In [46]:
city_spisok

['Москва', 'Санкт-Петербург', 'Казань']

# 3.4. Создайте дополнительную колонку со значением 0 или 1 в зависимости от часа заказа.

1 - если час пик (возьмите, например, 7,8,9,10,17,18,19,20), а 0 - все остальные часы.

Проверьте результат АВ-теста по отдельности на пиковых и не-пиковых заказах и сделайте выводы.

In [47]:
Dataset.head(10)

,id_view,time_view,id_group,id_city,nflag_order,name_city
0,10457162393,2023-01-17 13:56:06.118,0,3,1,Новосибирск
1,10457197478,2023-01-02 11:02:58.189,0,3,0,Новосибирск
2,10457135372,2023-01-15 10:03:58.011,0,3,0,Новосибирск
3,10457162971,2023-01-02 09:39:31.283,0,3,1,Новосибирск
4,10457132942,2023-01-23 19:18:03.516,0,3,0,Новосибирск
5,10457188717,2023-01-08 23:42:54.023,0,3,1,Новосибирск
6,10457156641,2023-01-15 06:08:14.919,1,3,1,Новосибирск
7,10457167469,2023-01-08 20:39:06.840,1,3,1,Новосибирск
8,10457125370,2023-01-15 22:47:00.255,1,3,1,Новосибирск
9,10457162002,2023-01-17 18:45:35.647,0,3,1,Новосибирск


In [91]:
Dataset['time_x'] = 0

In [101]:
from datetime import datetime

for i in range(0, len(Dataset)):
    hot_list = [7,8,9,10,17,18,19,20]
    date_time = Dataset['time_view'][i]
    hour = date_time.hour
    if hour in hot_list:
        Dataset['time_x'][i] = 1

C:\Users\Екатерина\AppData\Local\Temp\ipykernel_6256\1729576924.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset['time_x'][i] = 1


In [102]:
Dataset.head(20)

,id_view,time_view,id_group,id_city,nflag_order,name_city,time_x
0,10457162393,2023-01-17 13:56:06.118,0,3,1,Новосибирск,0
1,10457197478,2023-01-02 11:02:58.189,0,3,0,Новосибирск,0
2,10457135372,2023-01-15 10:03:58.011,0,3,0,Новосибирск,1
3,10457162971,2023-01-02 09:39:31.283,0,3,1,Новосибирск,1
4,10457132942,2023-01-23 19:18:03.516,0,3,0,Новосибирск,1
5,10457188717,2023-01-08 23:42:54.023,0,3,1,Новосибирск,0
6,10457156641,2023-01-15 06:08:14.919,1,3,1,Новосибирск,0
7,10457167469,2023-01-08 20:39:06.840,1,3,1,Новосибирск,1
8,10457125370,2023-01-15 22:47:00.255,1,3,1,Новосибирск,0
9,10457162002,2023-01-17 18:45:35.647,0,3,1,Новосибирск,1


In [ ]:
можно попробовать пробежаться по датасету по часам х и по часам не х

In [ ]:
проверяем отборы по пробочному времени

In [104]:
Dataset_probka = Dataset[Dataset['time_x']==1]
Dataset_ne_probka = Dataset[Dataset['time_x']==0]

In [105]:
group_test_p = Dataset_probka[Dataset_probka['id_group']==1]
group_control_p = Dataset_probka[Dataset_probka['id_group']==0]
city_spisok_chas_X = []
for i in Dataset['name_city'].unique():
    print('--------------------------------------------')
    print(i)
    t_stat, p_val = ttest_ind(group_test_p[group_test_p['name_city']==i]['nflag_order'], group_control_p[group_control_p['name_city']==i]['nflag_order'])
    print('значение статистического китерия - ', t_stat)
    print('P-значение (англ. P-value), p-уровень значимости, p-критерий (сила нулевой гипотезы)—', p_val)
    if p_val < 0.05:
        if p_val > 0:
            print('тест лучше чем контроль в пробку')
            print('новая механика лучше старой в час х')
        else:
            print('тест хуже чем контроль в час х')
            print('старая механика лучше новой в час икс')
        city_spisok_chas_X.append(i)
    else:
        print('p_val =', p_val, ' > 0.05')
        print('Нет статистически значимой разницы между старой и новой механиками даже в час х')
    print('--------------------------------------------')    

--------------------------------------------
Новосибирск
значение статистического китерия -  -0.03509071737172927
P-значение (англ. P-value), p-уровень значимости, p-критерий (сила нулевой гипотезы)— 0.9720136785031339
p_val = 0.9720136785031339  > 0.05
Нет статистически значимой разницы между старой и новой механиками даже в час х
--------------------------------------------
--------------------------------------------
Москва
значение статистического китерия -  6.790685972172624
P-значение (англ. P-value), p-уровень значимости, p-критерий (сила нулевой гипотезы)— 1.3449585112087216e-11
тест лучше чем контроль в пробку
новая механика лучше старой в час х
--------------------------------------------
--------------------------------------------
Санкт-Петербург
значение статистического китерия -  4.245735352177566
P-значение (англ. P-value), p-уровень значимости, p-критерий (сила нулевой гипотезы)— 2.3018931356634527e-05
тест лучше чем контроль в пробку
новая механика лучше старой в час х

In [106]:
city_spisok_chas_X

['Москва', 'Санкт-Петербург']

In [ ]:
А теперь в обычной жизни не в пробку

In [107]:
group_test_pn = Dataset_ne_probka[Dataset_ne_probka['id_group']==1]
group_control_pn = Dataset_ne_probka[Dataset_ne_probka['id_group']==0]
city_spisok_norm = []
for i in Dataset['name_city'].unique():
    print('--------------------------------------------')
    print(i)
    t_stat, p_val = ttest_ind(group_test_pn[group_test_pn['name_city']==i]['nflag_order'], group_control_pn[group_control_pn['name_city']==i]['nflag_order'])
    print('значение статистического китерия - ', t_stat)
    print('P-значение (англ. P-value), p-уровень значимости, p-критерий (сила нулевой гипотезы)—', p_val)
    if p_val < 0.05:
        if p_val > 0:
            print('тест лучше чем контроль в обычноек время')
            print('новая механика лучше старой в обычное время')
        else:
            print('тест хуже чем контроль в обычное время')
            print('старая механика лучше новой в обычное время')
        city_spisok_norm.append(i)
    else:
        print('p_val =', p_val, ' > 0.05')
        print('Нет статистически значимой разницы между старой и новой механиками даже в час х')
    print('--------------------------------------------')    

--------------------------------------------
Новосибирск
значение статистического китерия -  1.9033169219042654
P-значение (англ. P-value), p-уровень значимости, p-критерий (сила нулевой гипотезы)— 0.05713158719650457
p_val = 0.05713158719650457  > 0.05
Нет статистически значимой разницы между старой и новой механиками даже в час х
--------------------------------------------
--------------------------------------------
Москва
значение статистического китерия -  4.291499026558841
P-значение (англ. P-value), p-уровень значимости, p-критерий (сила нулевой гипотезы)— 1.8030823524055564e-05
тест лучше чем контроль в обычноек время
новая механика лучше старой в обычное время
--------------------------------------------
--------------------------------------------
Санкт-Петербург
значение статистического китерия -  3.407489376223579
P-значение (англ. P-value), p-уровень значимости, p-критерий (сила нулевой гипотезы)— 0.0006634181415393023
тест лучше чем контроль в обычноек время
новая механи

In [108]:
city_spisok_norm

['Москва', 'Санкт-Петербург', 'Казань', 'Архангельск']

## Задание 3.5
Опишите ваши результаты. Ваши выводы можно оформить внутри тетрадки .ipynb

Позитивные изменеия при смене механики оплаты происходят в Санкт-Петербурге и Москве в обучное время и даже в часы Х, так же в обычное врме вне часов пик похитивные изменения показывают Казань и Астрахань, но в целом по городам если оценивать и время пробок и время обычное то Астрахань тоже можно не рассматривать